# CrewAI DeepLearning.AI Course
## Example 1 - Technical Article Writer, using Agents to Research and Write an Article

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
from crewai import Agent, Task, Crew

In [11]:
import os
openai_api_key = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Focusing on Role Playing, Focus and Cooperation
* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

- Allowing an agent to delegate work and ask questions does not necessarily imply that it will do it everytime, but that it will do if it chooses to or thinks that is needed.

In [12]:
# Creating a support agent
support_agent = Agent(
    role="Senior Support Representative", # keyword, senior, to show that we want more 'polished' responses
	goal="Be the most friendly and helpful " # will play a piece on every step
        "support representative in your team",
	backstory=( 
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [13]:
# Creating our support QA agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    allow_delegation=True # if you don't set it, defualts to True
)

## Design Patterns with Crew:
* A common design pattern we see with crew is the quality assurance pattern. Where one agent does the task, the final QA agent reviews the task, and it can do a final pass to see if what it does was good. We can also have it delegate if it finds things wrong, and give it back to the original agent to fix what it finds wrong.

## Tools, Guardrails and Memory

### Tools
- Import CrewAI tools

In [14]:
# Serper integrates with Serper which is an external service to search Google
# The scraper accesses the website and extracts the data.
# Then the search tool is actually doing RAG over the website.
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool() # allows us to scrape any URL it finds fit
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

If you want to make it more specific, just to search this specific URL:
```Python
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/" # just allows this URL
)
```

In [23]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/core-concepts/Crews/#crew-output-methods-and-properties" 
) # just allows this URL

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [24]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool], # an array with all the tools you want for this task
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [25]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)
# purely assessing the response since it cannot access the docs


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [26]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

2024-07-28 11:03:39,704 - 8389712064 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [27]:
# We need to pass the inputs to override the interpolation variables:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [2024-07-28 11:03:45][DEBUG]: == Working Agent: Senior Support Representative
 [2024-07-28 11:03:45][INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to gather specific information about how to add memory to a Crew in crewAI to provide a comprehensive and accurate response to Andrew Ng from DeepLearningAI.

Action: Read website content  
Action Input: {}  
 

crewAI Crews - crewAI
 Skip to content
 crewAI
 crewAI Crews
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Processes
 Crews
 Crews
 Table of con